In [ ]:
import os
import json
import subprocess
import uuid
import IPython.display as ipd
from playlist_copilot.utils.smp_utils import get_track_info

In [ ]:
# Function to read .m3u file and extract MP3 file paths
def read_m3u(file_path):
    mp3_file_paths = []

    with open(file_path, 'r') as file:
        for line in file:
            # Skip empty lines and lines that start with '#'
            if line.strip() and not line.startswith('#'):
                mp3_file_paths.append(line.strip())

    return mp3_file_paths

# Function to create and display an audio player for each MP3 file
def play_mp3_files(file_paths):
    for path in file_paths:
        track_id = os.path.splitext(os.path.basename(path))[0]
        track_info = get_track_info(f'spotify:track:{track_id}')
        artist = track_info['artist']
        title = track_info['title']
        album = track_info['album']
        print(f"{artist} | {title} | {album}")
        display(ipd.Audio(path))

# Function to display the songs selected and create a json using which audio can be played
def print_playlist_songnames(file_paths, filename):
    json_for_audio_playback = []
    n=1
    for path in file_paths:
        track_id = os.path.splitext(os.path.basename(path))[0]
        track_info = get_track_info(f'spotify:track:{track_id}')
        track_json = {}
        artist = track_info['artist']
        title = track_info['title']
        album = track_info['album']
        track_json['sequence'] = n
        track_json['title'] = title
        track_json['artist'] = artist
        track_json['album'] = album
        track_json['audio_filepath'] = path
        print(f"{artist} | {title} | {album}")
        json_for_audio_playback.append(track_json)
        n+=1
    with open(f'{filename}.json', 'w') as file:
        json.dump(json_for_audio_playback, file, indent=4)

In [ ]:
# Pop
# Taylor Swift | I Knew You Were Trouble. | Red
test_1 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/audio/72jCZdH0Lhg93z6Z4hBjgj.mp3'

# Folk Rock / Alternative Rock
# Mumford & Sons | White Blank Page | Sigh No More
test_2 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/audio/14rApQTKycs9gNg7hjm9m7.mp3'

# Electronic
# Skrillex | Scary Monsters And Nice Sprites - Noisia Remix | Scary Monsters and Nice Sprites
test_3 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/audio/5bPokvOlgW9dRySVnVmyJ9.mp3'

# Hip Hop
# Sage The Gemini | Don't You | Remember Me
test_4 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/test_subset/4FOv3s8nyCctxcgstQwtFj.mp3'

# Instrumental music (piano and cello)
# The Piano Guys | Bring Him Home | The Piano Guys
test_5 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/test_subset/2N6I3slzcRXW7pUw2x30qL.mp3'


In [ ]:
# USE CASE 1:= Starting from a single seed song

# Choose parameters and song
input_file = test_5

# Additional parameters, if needed
num_songs = '10'  # Number of songs in the playlist
noise = '0'   # Amount of noise in the playlist (default 0)
lookback = '4' # Amount of lookback in the playlist (default 3)

# Path to Python script
script = 'Deej-A.I.py'

# Arguments for the script
mp3tovec = 'mp3tovec'
output_file = str(uuid.uuid4()) + '.m3u'

# Construct the command as a list
command = ['python', script, 'Pickles_test', mp3tovec, '--playlist', output_file,
           '--nsongs', num_songs, '--noise', noise, '--lookback', lookback,
           '--inputsong', input_file]

# Execute the command
subprocess.run(command)

m3u_file_path = output_file

# Read the .m3u file and get the MP3 file paths
mp3_file_paths = read_m3u(m3u_file_path)
print_playlist_songnames(mp3_file_paths, os.path.splitext(m3u_file_path)[0])

In [ ]:
# Use this cell to listen to the audios

# Call the function with the placeholder paths
mp3_file_paths = read_m3u(m3u_file_path)
play_mp3_files(mp3_file_paths)